In [55]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext

In [56]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [57]:
# 前処理と単語分割をまとめた関数を作成
import re
import string
from utils.bert import BertTokenizer
# フォルダ「utils」のbert.pyより


def preprocessing_text(text):
    '''IMDbの前処理'''
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


# 単語分割用のTokenizerを用意
tokenizer_bert = BertTokenizer(
    vocab_file="./vocab/bert-base-uncased-vocab.txt", do_lower_case=True)


# 前処理と単語分割をまとめた関数を定義
# 単語分割の関数を渡すので、tokenizer_bertではなく、tokenizer_bert.tokenizeを渡す点に注意
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)  # tokenizer_bert
    return ret


In [58]:
max_length = 256

TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", eos_token="[SEP]", pad_token='[PAD]', unk_token='[UNK]')
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [59]:
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

In [60]:
from utils.bert import BertTokenizer, load_vocab

vocab_bert, ids_to_tokens_bert = load_vocab(
    vocab_file="./vocab/bert-base-uncased-vocab.txt")

TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert

In [61]:
batch_size = 32  # BERTでは16、32あたりを使用する

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=batch_size, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}

In [62]:
# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[  101, 11590, 11821,  ...,     0,     0,     0],
        [  101,  1045,  2310,  ...,     0,     0,     0],
        [  101,  2023,  2003,  ...,     0,     0,     0],
        ...,
        [  101,  2116, 19046,  ..., 17289, 13157,   102],
        [  101,  2417,  2600,  ...,  1037,  6659,   102],
        [  101,  2043,  1045,  ...,     0,     0,     0]]), tensor([154, 241, 102, 224, 111, 256, 106, 203, 256,  58, 256, 146, 230,  71,
        256, 256, 117, 217, 209,  91, 256, 253, 216, 163, 256, 256, 137, 256,
        230, 256, 256, 101]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 1, 1, 1])


In [63]:
batch.Text[0].size()

torch.Size([32, 256])

In [64]:
# ミニバッチの1文目を確認してみる
text_minibatch_1 = (batch.Text[0][1]).numpy()

# IDを単語に戻す
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)


['[CLS]', 'i', 've', 'now', 'seen', 'this', 'one', 'about', '10', 'times', ',', 'so', 'there', 'must', 'be', 'something', 'about', 'it', 'i', 'like', '50', 's', 'us', 'sci', 'fi', 'movies', 'were', 'pretty', 'much', 'a', 'mixed', 'bunch', 'they', 'were', 'either', 'intelligent', '##ly', 'made', 'and', 'or', 'thought', 'pro', '##voking', 'or', 'cheap', 'and', 'laugh', '##able', 'cheese', '.', 'forbidden', 'planet', 'is', 'a', 'bit', 'of', 'both', ',', 'but', 'in', 'that', 'ra', '##rity', 'for', 'the', 'genre', ',', 'colour', '.', 'it', 'also', 'had', 'a', 'head', 'start', 'with', 'the', 'script', 'although', 'shakespeare', 'might', 'not', 'have', 'recognised', 'it', ',', 'it', 'was', 'based', 'on', 'his', 'timeless', 'play', 'and', 'thus', 'guaranteed', 'a', 'certain', 'amount', 'of', 'longevity', 'itself', 'if', 'made', 'well', '.', 'it', 's', 'the', 'story', 'of', 'one', 'mans', 'murderous', 'id', 'artificial', '##ly', 'mag', '##nified', 'infinitely', 'by', 'machines', 'a', 'dead', 'r

In [65]:
len(text)

256

In [66]:
from utils.bert import get_config, BertModel, set_learned_params

# モデル設定のJOSNファイルをオブジェクト変数として読み込みます
config = get_config(file_path="./weights/bert_config.json")

# BERTモデルを作成します
net_bert = BertModel(config)

# BERTモデルに学習済みパラメータセットします
net_bert = set_learned_params(
    net_bert, weights_path="./weights/pytorch_model.bin")


bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.output

In [67]:
class BertForIMDb(nn.Module):
    
    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()
        self.bert = net_bert
        self.cls = nn.Linear(in_features=768, out_features=2)
        
        with torch.no_grad():
            nn.init.normal_(self.cls.weight, std=0.02)
            nn.init.normal_(self.cls.bias, 0)
    
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        if attention_show_flg == True:
            '''attention_showのときは、attention_probsもリターンする'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
        out = self.cls(vec_0)
        
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out

In [68]:
nn.init.normal

<function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>

In [69]:
# モデル構築
net = BertForIMDb(net_bert)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


ネットワーク設定完了


In [70]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for name, param in net.named_parameters():
    param.requires_grad = False

# 2. 最後のBertLayerモジュールを勾配計算ありに変更
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for name, param in net.cls.named_parameters():
    param.requires_grad = True


In [71]:
# 最適化手法の設定

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 5e-5}
], betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算


In [72]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # 開始時刻を保存
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
#                 print("inputs size: ",inputs.size())
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BertForIMDbに入力
                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_flg=False)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(), duration, acc))
                            t_iter_start = time.time()

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()

    return net


In [73]:
# 学習・検証を実行する。1epochに20分ほどかかります
num_epochs = 2
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 0.7260 || 10iter: 4.0281 sec. || 本イテレーションの正解率：0.46875
イテレーション 20 || Loss: 0.6372 || 10iter: 4.0601 sec. || 本イテレーションの正解率：0.65625
イテレーション 30 || Loss: 0.6484 || 10iter: 4.0846 sec. || 本イテレーションの正解率：0.5625
イテレーション 40 || Loss: 0.6375 || 10iter: 3.8598 sec. || 本イテレーションの正解率：0.65625
イテレーション 50 || Loss: 0.6173 || 10iter: 3.9114 sec. || 本イテレーションの正解率：0.625
イテレーション 60 || Loss: 0.6189 || 10iter: 4.0664 sec. || 本イテレーションの正解率：0.6875
イテレーション 70 || Loss: 0.4001 || 10iter: 3.9118 sec. || 本イテレーションの正解率：0.9375
イテレーション 80 || Loss: 0.4774 || 10iter: 3.9125 sec. || 本イテレーションの正解率：0.78125
イテレーション 90 || Loss: 0.4081 || 10iter: 3.9042 sec. || 本イテレーションの正解率：0.875
イテレーション 100 || Loss: 0.5042 || 10iter: 3.9093 sec. || 本イテレーションの正解率：0.75
イテレーション 110 || Loss: 0.2994 || 10iter: 3.9297 sec. || 本イテレーションの正解率：0.875
イテレーション 120 || Loss: 0.3203 || 10iter: 3.9038 sec. || 本イテレーションの正解率：0.84375
イテレーション 130 || Loss: 0.4903 || 10iter: 3.9103 sec. || 本イテレーションの正解率：0.84375
イテレーション 140 |

イテレーション 480 || Loss: 0.3590 || 10iter: 3.2516 sec. || 本イテレーションの正解率：0.84375
イテレーション 490 || Loss: 0.3163 || 10iter: 3.2681 sec. || 本イテレーションの正解率：0.84375
イテレーション 500 || Loss: 0.2526 || 10iter: 3.2571 sec. || 本イテレーションの正解率：0.90625
イテレーション 510 || Loss: 0.2097 || 10iter: 3.2785 sec. || 本イテレーションの正解率：0.9375
イテレーション 520 || Loss: 0.2017 || 10iter: 3.2649 sec. || 本イテレーションの正解率：0.90625
イテレーション 530 || Loss: 0.1241 || 10iter: 3.3017 sec. || 本イテレーションの正解率：0.96875
イテレーション 540 || Loss: 0.1619 || 10iter: 3.2683 sec. || 本イテレーションの正解率：0.90625
イテレーション 550 || Loss: 0.1840 || 10iter: 3.2596 sec. || 本イテレーションの正解率：0.875
イテレーション 560 || Loss: 0.3735 || 10iter: 3.2426 sec. || 本イテレーションの正解率：0.875
イテレーション 570 || Loss: 0.2478 || 10iter: 3.2486 sec. || 本イテレーションの正解率：0.9375
イテレーション 580 || Loss: 0.2075 || 10iter: 3.2302 sec. || 本イテレーションの正解率：0.9375
イテレーション 590 || Loss: 0.2498 || 10iter: 3.2314 sec. || 本イテレーションの正解率：0.90625
イテレーション 600 || Loss: 0.2645 || 10iter: 3.2335 sec. || 本イテレーションの正解率：0.875
イテレーション 610 || Loss: 0.2173 || 10i

In [74]:
# 学習したネットワークパラメータを保存します
save_path = './weights/bert_fine_tuning_IMDb.pth'
torch.save(net_trained.state_dict(), save_path)


In [75]:
# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(test_dl):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForIMDbに入力
        outputs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                              output_all_encoded_layers=False, attention_show_flg=False)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dl.dataset), epoch_acc))


100%|██████████| 782/782 [04:40<00:00,  2.79it/s]

テストデータ25000個での正解率：0.9027


In [77]:
# batch_sizeを64にしたテストデータでDataLoaderを作成
batch_size = 64
test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)


In [78]:
# BertForIMDbで処理

# ミニバッチの用意
batch = next(iter(test_dl))

# GPUが使えるならGPUにデータを送る
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # ラベル

outputs, attention_probs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                                       output_all_encoded_layers=False, attention_show_flg=True)

_, preds = torch.max(outputs, 1)  # ラベルを予測


In [79]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {};color:black"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # ラベル
    pred = preds[index]  # 予測

    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    # Self-Attentionの重みを可視化。Multi-Headが12個なので、12種類のアテンションが存在
    for i in range(12):

        # indexのAttentionを抽出と規格化
        # 0単語目[CLS]の、i番目のMulti-Head Attentionを取り出す
        # indexはミニバッチの何個目のデータかをしめす
        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()

        html += '[BERTのAttentionを可視化_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            # 単語が[SEP]の場合は文章が終わりなのでbreak
            if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
            html += highlight(tokenizer_bert.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"

    # 12種類のAttentionの平均を求める。最大値で規格化
    all_attens = attens*0  # all_attensという変数を作成する
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    html += '[BERTのAttentionを可視化_ALL]<br>'
    for word, attn in zip(sentence, attens):

        # 単語が[SEP]の場合は文章が終わりなのでbreak
        if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break

        # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
        html += highlight(tokenizer_bert.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
    html += "<br><br>"

    return html


In [80]:
from IPython.display import HTML

index = 3  # 出力させたいデータ
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力


In [81]:
index = 61  # 出力させたいデータ
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力
